In [55]:
# Importing necessary libraries
import warnings
warnings.filterwarnings('ignore')

from tensorflow import keras

from keras.layers import Input, Lambda, Dense, Flatten

from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt



In [56]:
# Find and load train and test set directories
#resize the images
image_size=[224,224]
train_dir = "/Users/saman/OneDrive/Documents/EFREI/Datacamp/chest_xray/train"
test_dir="/Users/saman/OneDrive/Documents/EFREI/Datacamp/chest_xray/test"

In [57]:
#pre_trained model
vgg = VGG16(input_shape=image_size + [3], weights='imagenet', include_top=False)

In [58]:
for layer in vgg.layers:
    layer.trainable = False

In [59]:
# getting number of output classes
file = glob('/Users/saman/OneDrive/Documents/EFREI/Datacamp/chest_xray/train/*')
# our layers
x = Flatten()(vgg.output)

In [60]:
prediction = Dense(len(file), activation='softmax')(x)
# Create a Model object
base_model = Model(inputs=vgg.input, outputs=prediction)
# See the layer and parameter summary
base_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [61]:
# tell the model what cost and optimization method to use
base_model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [62]:
from keras.preprocessing.image import ImageDataGenerator

In [63]:
#Image data generator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

# Making sure that we provide the same target size as initialied for the image size

training_set = train_datagen.flow_from_directory('/Users/saman/OneDrive/Documents/EFREI/Datacamp/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')



test_set = test_datagen.flow_from_directory('/Users/saman/OneDrive/Documents/EFREI/Datacamp/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 5232 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [116]:
#fit the model
r = base_model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=2,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/2
524/524 [==============================] - 722s 1s/step - loss: 0.2262 - accuracy: 0.9247 - val_loss: 0.2469 - val_accuracy: 0.9215
Epoch 2/2
524/524 [==============================] - 856s 2s/step - loss: 0.1712 - accuracy: 0.9459 - val_loss: 0.3051 - val_accuracy: 0.9199


In [117]:
r.history

{'loss': [0.22620749473571777, 0.17115826904773712],
 'accuracy': [0.9246941804885864, 0.9459097981452942],
 'val_loss': [0.24689170718193054, 0.30510619282722473],
 'val_accuracy': [0.9214743375778198, 0.9198718070983887]}

In [118]:
import tensorflow as tf
from keras.models import load_model

base_model.save('chest_xray.h5')

In [119]:
# Evaluate the model 
print("Evaluate test data")
results = base_model.evaluate(test_set, batch_size=16)
print("Test loss, Test accuracy:", results)

Evaluate on test data
63/63 [==============================] - 62s 982ms/step - loss: 0.3051 - accuracy: 0.9199
test loss, test acc: [0.3051062226295471, 0.9198718070983887]


In [51]:
from keras.models import load_model
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import load_img
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.metrics import classification_report,confusion_matrix

In [42]:
base_model=load_model('chest_xray.h5')

In [43]:
#test our model
image=load_img(r'C:\Users\saman\OneDrive\Documents\EFREI\Datacamp\chest_xray\train\PNEUMONIA\BACTERIA-92115-0001.jpeg',target_size=(224,224))

In [44]:
x=img_to_array(image)

In [45]:
x=np.expand_dims(x, axis=0)

In [46]:
image_data=preprocess_input(x)

In [48]:
categories=base_model.predict(image_data)
print(categories)

1/1 [==============================] - 2s 2s/step
[[8.700746e-14 1.000000e+00]]


In [35]:
print(categories[0])

[2.6447812e-21 1.0000000e+00]


In [30]:
result=int(categories[0][0])

In [31]:
if result==0:
    print("PNEUMONIA")
else:
    print("Normal")

Person is Affected By PNEUMONIA
